Goal: Evaluate the behaviour of the sub of primes

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.multioutput import RegressorChain
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# Load the data
f = open("../data/rawRecords_64_256000.csv")
f.readline()  # skip the header
data = np.loadtxt(f, delimiter=',')

rawX = data[:, 0].reshape(-1, 1)
rawY = data[:, 1:]
# add sub column
subColumn = np.subtract(rawY[:, 1],rawY[:, 0]).reshape(-1, 1)
# sumColumn = np.sum(rawY,1).reshape(-1, 1) a sum does not seem to yield a better estimation but worse for some reason.

# newX = np.append(rawX, subColumn, 1)
# newY = rawY
newX = rawX
newY = subColumn

Scale/Transform

In [ ]:
xScaler = MaxAbsScaler()
xScaler.fit(newX)
transformedRawX = xScaler.transform(newX)
yScaler = MaxAbsScaler()
yScaler.fit(newY)
transformedRawY = yScaler.transform(newY)

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(transformedRawX, transformedRawY)

In [ ]:
# plot sub relation
import seaborn as sns
import pandas as pd

df = pd.read_csv('../data/rawRecords_100_256000.csv')
df2 = df.assign(subPrimes=lambda r: r.BiggerPrime - r.SmallerPrime)
sns.pairplot(df2, diag_kind='kde')

Train

In [ ]:
regressor = MLPRegressor()
regressor.fit(X_train, y_train)

In [ ]:
print('regressor.score(x_test, y_test)=' + str(regressor.score(X_test, y_test)))

x_test_zero = transformedRawX[-2:-1]
y_test_zero = transformedRawY[-2:-1]
y_pred = regressor.predict(x_test_zero)
print(f'x_test_zero={str(x_test_zero)} y_pred_zero={str(y_pred)} y_test_zero={str(y_test_zero)}')

y_pred_converted = yScaler.inverse_transform(y_pred[0].reshape(1, -1))
y_pred_converted_0 = int(y_pred_converted[0][0])
y_pred_converted_1 = int(y_pred_converted[0][1])
print(f'y_pred_converted={str(y_pred_converted_0)}, {str(y_pred_converted_1)}')


y_pred = regressor.predict(X_test)
plt.scatter(X_test[:, 0], y_test[:, 1], color="black")
plt.scatter(X_test[:, 0], y_pred[:, 1], color="blue")
plt.show()

plt.scatter(X_test[:, 0], X_test[:, 1], color="black")
plt.show()

plt.plot(regressor.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()

In [ ]:
# %matplotlib widget
plt.scatter(X_test[:, 1], y_test[:, 1], color="black")
plt.scatter(X_test[:, 1], y_pred[:, 1], color="blue")
plt.show()